# 第10章: 事前学習済み言語モデル (GPT型)


本章では, GPT型 (Transformerのデコーダ型) の事前学習済みモデルを利用して, 言語生成, 評判分析器 (ポジネガ分類器) の構築, ファインチューニング, 強化学習などに取り組む.

```{warning}
本章は, `code-cell` ではなく, Markdown のコードブロック内にコードを記述しているため, Google Colab上で直接実行できません.
```

```{note}
**ToDo:** 98, 99は`gpt2-medium`から`HuggingFaceH4/zephyr-7b-beta`に変更する.
```

## 90. 次単語予測


"The movie was full of"に続くトークン (トークン列ではなく一つのトークンであることに注意せよ) として適切なもの上位10個と, その確率 (尤度) を求めよ.ただし, 言語モデルへのプロンプトがどのようなトークン列に変換されたか, 確認せよ.

```python
from transformers import GPT2Tokenizer, GPT2LMHeadModel, set_seed
import torch

set_seed(1234)

model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

text = "The movie was full of"
input_ids = tokenizer.encode(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

next_token_logits = logits[0, -1]
probs = torch.softmax(next_token_logits, dim=-1)

k = 10
top_probs, top_indices = torch.topk(probs, k)

for i in range(k):
    decoded_text = tokenizer.decode([top_indices[i]])
    print(f"{decoded_text}: {top_probs[i]}")
```

```bash
 great: 0.02309427782893181
 references: 0.013511945493519306
 action: 0.013043278828263283
 moments: 0.012449497357010841
 the: 0.01186001393944025
 characters: 0.008720042183995247
 these: 0.007216328755021095
 surprises: 0.006894332356750965
 fun: 0.006525978911668062
 them: 0.006154114380478859
```

## 91. 続きのテキストの予測


"The movie was full of"に続くテキストを複数予測せよ.このとき, デコーディングの方法や温度パラメータ (temperature) を変えながら, 予測される複数のテキストの変化を観察せよ.

```python
from transformers import pipeline, set_seed

set_seed(1234)
generator = pipeline('text-generation', model='gpt2-medium')

text = "The movie was full of"

for t in [0.2, 0.4, 0.7, 0.9]:
    outputs = generator(text, max_length=30, num_return_sequences=1, temperature=t)
    print(f'Temp={t}: {outputs[0]["generated_text"]}')
```

```bash
Temp=0.2: The movie was full of great moments, but it was also a bit of a letdown. The film was a bit of a letdown.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Temp=0.4: The movie was full of action sequences that were very exciting to watch. The movie was very well done, and I think it was a great movie.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Temp=0.7: The movie was full of all the things I love about this country: the rich, the black, the working class," said D'Amato.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Temp=0.9: The movie was full of action and violence. The scene in the prison in which Lee and Davis are talking and the scene in which Lee kills John,
```

## 92. 予測されたテキストの確率を計算

"The movie was full of"に続くテキストを予測し, 生成された各単語の尤度を表示せよ (生成されるテキストが長いと出力が読みにくくなるので, 適当な長さで生成を打ち切るとよい) .

## 93. パープレキシティ

適当な文を準備して, 事前学習済み言語モデルでパープレキシティを測定せよ.例えば, 

- The movie was full of surprises
- The movies were full of surprises
- The movie were full of surprises
- The movies was full of surprises


の4文に対して, パープレキシティを測定して観察せよ (最後の2つの文は故意に文法的な間違いを入れた).

```python
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import math

def calc_ppl(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = input_ids[..., 1:].contiguous()

    loss_fn = torch.nn.CrossEntropyLoss(reduction="none")
    loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

    return math.exp(loss.mean().item())

model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

sentences = [
    "The movie was full of surprises",
    "The movies were full of surprises",
    "The movie were full of surprises",
    "The movies was full of surprises"
]

for sentence in sentences:
    print(f'{sentence}: {calc_ppl(sentence)}')
```

```python


```bash
The movie was full of surprises: 89.45385588749441
The movies were full of surprises: 164.8886480442875
The movie were full of surprises: 324.1063371099563
The movies was full of surprises: 388.4464576262505
```

## 94. チャットテンプレート


"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため, チャットテンプレートを適用し, 言語モデルに与えるべきプロンプトを作成せよ.また, そのプロンプトに対する応答を生成し, 表示せよ.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch

set_seed(1234)    
    
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

chat = [
    {"role": "system", "content": "You are an excellent assistant."},
    {"role": "user", "content": "What do you call a sweet eaten after dinner?"}
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generate_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)
```

```bash
<|system|>
You are an excellent assistant.
<|user|>
What do you call a sweet eaten after dinner?
<|assistant|>
A dessert or a dessert item, such as cake, pie, cookies, ice cream, or fruit with a sweet sauce. The specific term may vary based on the type of sweet being referred to.
```

## 95. マルチターンのチャット


問題94で生成された応答に対して, 追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ.また, その時に言語モデルに与えるプロンプトを確認せよ.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch

set_seed(1234)  

def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return output[0]

def extract_assistant_reply(text):
    text = text.split("<|assistant|>")
    return text[-1].replace("\n", "").strip()

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

chat = [
    {"role": "system", "content": "You are an excellent assistant."},
    {"role": "user", "content": "What do you call a sweet eaten after dinner?"}
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generate_prompt=True)
raw_text = generate_text(prompt)
decoded_text = tokenizer.decode(raw_text, skip_special_tokens=True)

chat.append({"role": "assistant", "content": extract_assistant_reply(decoded_text)})
chat.append({"role": "user", "content": "Please give me the plural form of the word with its spelling in reverse order."})

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generate_prompt=True)
raw_text = generate_text(prompt)
decoded_text = tokenizer.decode(raw_text, skip_special_tokens=True)
print(decoded_text)
```

```bash
<|system|>
You are an excellent assistant.
<|user|>
What do you call a sweet eaten after dinner?
<|assistant|>
A dessert or a dessert item, such as cake, pie, cookies, ice cream, or fruit with a sweet sauce. The specific term may vary based on the type of sweet being referred to.
<|user|>
Please give me the plural form of the word with its spelling in reverse order.
<|assistant|>
If the word ends in "s," simply add an "es" to the end. However, if the word ends in "ch," "sh," "x," or "s" preceded by a consonant, you add "es" and change the "s" to "es" as well. For example:

- Churches (churches)
- Boxes (boxes)
-Ches (chess)
-Kisses (kisses)

In reverse order, the plural forms would be:

- Esreuq siht noitavT (desserts)
- EttorC siht noitavT (cookies)
- EhT roE siht noitavT (cakes)
- EhT roE siht noitavT (pies)
- EhT roE siht noitavT (ice cream)
- EhT roE siht noitavT (fruit)
- EhT roE siht noitavT (sauces)

Note: The reverse spelling is just for fun and is not a commonly used method for spelling words in English.
```

## 96. プロンプトによる感情分析


事前学習済み言語モデルで感情分析を行いたい.テキストを含むプロンプトを事前学習済み言語モデルに与え, (ファインチューニングは行わずに) テキストのポジネガを予測するという戦略で, [SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm

def generate_text(prompt, tokenizer, model, device, max_new_tokens):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return output[0]

def extract_assistant_reply(text):
    text = text.rstrip()
    return text[-1]

def generate_examples(df, k, random_state=1234):
    pos_examples = df[df["label"] == 1].sample(n=k, random_state=random_state)
    neg_examples = df[df["label"] == 0].sample(n=k, random_state=random_state)
    examples = list(zip(pos_examples["sentence"], pos_examples["label"])) + list(zip(neg_examples["sentence"], neg_examples["label"]))
    
    prompt = []
    for sentence, label in examples:
        prompt.append({"role": "user", "content": sentence})
        prompt.append({"role": "assistant", "content": str(label)})
        
    return prompt

set_seed(1234)

train_df = pd.read_table("../ch07/SST-2/train.tsv", delimiter="\t")
dev_df = pd.read_table("../ch07/SST-2/dev.tsv", delimiter="\t")

model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

tokens = tokenizer.tokenize("<|assistant|>\n1")
max_tokens = len(tokens)

device = model.device

preds = []
valid_labels = []
skipped_count = 0
examples = generate_examples(train_df, k=10)
for i in tqdm(range(len(dev_df)), desc="Progress"):
    chat = [
        {"role": "system", "content": "You are a professional in emotional analysis. Please judge the following sentences positively or negatively. If it is positive, output 1, if it is negative, output 0."},
        *examples,
        {"role": "user", "content": dev_df["sentence"][i]}
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generate_prompt=True)
    raw_output = generate_text(prompt, tokenizer, model, device, max_tokens)
    decoded_text = tokenizer.decode(raw_output, skip_special_tokens=True)
    pred = extract_assistant_reply(decoded_text)

    if pred in {"0", "1"}:
        preds.append(int(pred))
        valid_labels.append(int(dev_df["label"][i]))
    else:
        skipped_count += 1

acc = accuracy_score(valid_labels, preds)
print(f"Acc: {acc}")
print(f"Skipped samples: {skipped_count}")
```

```bash
Acc: 0.9380733944954128
Skipped samples: 0
```

## 97. 埋め込みに基づく感情分析


事前学習済み言語モデルでテキストをベクトルで表現 (エンコード) し, そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ.

```python
from transformers import AutoTokenizer, GPT2Model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import numpy as np

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)

set_seed(123)

train_df = pd.read_table("../ch07/SST-2/train.tsv", delimiter="\t")
dev_df = pd.read_table("../ch07/SST-2/dev.tsv", delimiter="\t")

class SSTDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['sentence']
        label = int(self.data.iloc[idx]['label'])
        
        encoding = self.tokenizer(
            sentence,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

class SentimentClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(SentimentClassifier, self).__init__()
        self.encoder = GPT2Model.from_pretrained("gpt2-medium")
        hidden_size = self.encoder.config.n_embd
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = input_ids.shape[0]
        selected_hidden_states = last_hidden_states[torch.arange(batch_size), sequence_lengths]
        logits = self.classifier(selected_hidden_states)
        return logits

def train_model(model, train_loader, val_loader, device, epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-5)
    
    best_val_accuracy = 0.0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
        train_loss /= len(train_loader)

        model.eval()
        val_preds = []
        val_true = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                _, preds = torch.max(outputs, 1)
                
                val_preds.extend(preds.cpu().tolist())
                val_true.extend(labels.cpu().tolist())
        
        val_accuracy = accuracy_score(val_true, val_preds)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), "best_gpt2_medium_sentiment_model.pt")
            print(f"Model saved with accuracy: {best_val_accuracy:.4f}")
    
    return best_val_accuracy

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")
    tokenizer.pad_token = tokenizer.eos_token

    train_dataset = SSTDataset(train_df, tokenizer)
    val_dataset = SSTDataset(dev_df, tokenizer)

    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    model = SentimentClassifier()
    model.to(device)

    best_accuracy = train_model(model, train_loader, val_loader, device, epochs=3)
    print(f"Training completed with best validation accuracy: {best_accuracy:.4f}")

if __name__ == "__main__":
    main()

```

```python
Epoch 1/3 [Train]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2105/2105 [36:21<00:00,  1.04s/it]
Epoch 1/3 [Val]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  3.01it/s]
Epoch 1/3, Train Loss: 0.2461, Val Accuracy: 0.9300
Model saved with accuracy: 0.9300
Epoch 2/3 [Train]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2105/2105 [35:58<00:00,  1.03s/it]
Epoch 2/3 [Val]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  3.03it/s]
Epoch 2/3, Train Loss: 0.1375, Val Accuracy: 0.9358
Model saved with accuracy: 0.9358
Epoch 3/3 [Train]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2105/2105 [35:38<00:00,  1.02s/it]
Epoch 3/3 [Val]: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:09<00:00,  3.03it/s]
Epoch 3/3, Train Loss: 0.1015, Val Accuracy: 0.9369
Model saved with accuracy: 0.9369
Training completed with best validation accuracy: 0.9369
```

## 98. ファインチューニング


問題96のプロンプトに対して, 正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, set_seed
import torch
from datasets import load_dataset
from copy import deepcopy
from sklearn.metrics import accuracy_score
from tqdm import tqdm

set_seed(1234)

def tokenize_function(examples):
    full_texts = [
        "Sentence: " + sentence + "\nLabel: " + str(label)
        for sentence, label in zip(examples["sentence"], examples["label"])
    ]
    tokenized = tokenizer(full_texts, truncation=True, padding="max_length", max_length=128)
    
    labels = deepcopy(tokenized["input_ids"])
    
    prompt_texts = [
        "Sentence: " + sentence + "\nLabel:" 
        for sentence in examples["sentence"]
    ]
    prompt_tokenized = tokenizer(prompt_texts, truncation=True, padding="max_length", max_length=128)
    
    for i, prompt_ids in enumerate(prompt_tokenized["input_ids"]):
        prompt_len = len(prompt_ids)
        labels[i][:prompt_len] = [-100] * prompt_len
    
    tokenized["labels"] = labels
    return tokenized

def train_model(model, datasets, training_args, data_collator=None, output_dir=None):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets["train"],
        eval_dataset=datasets["validation"],
        data_collator=data_collator
    )
    
    trainer.train()
    
    if output_dir is not None:
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

def eval_model(model, datasets, max_new_tokens=2, max_length=None, num_return_sequences=1, skip_special_tokens=True):
    input_texts = [f"Sentence: {ex['sentence']}\nLabel:" for ex in datasets]
    labels = [str(ex['label']) for ex in datasets]
    preds = []
    for input_text in tqdm(input_texts, desc="Evaluating"):
        inputs = tokenizer(input_text, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, max_length=max_length, num_return_sequences=num_return_sequences)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=skip_special_tokens, eos_token_id=tokenizer.eos_token_id)
        label_start = generated_text.find("Label:") + len("Label:")
        pred = generated_text[label_start:].strip().split()[0]
        preds.append(pred)
        
    acc = accuracy_score(labels, preds)
    print(f"Acc: {acc}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset("csv", data_files={"train": "../ch07/SST-2/train.tsv", "validation": "../ch07/SST-2/dev.tsv"}, delimiter='\t')
tokenized_datasets = dataset.map(tokenize_function, batched=True)
    
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    eval_accumulation_steps=10
)
   
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model_output_dir = "./results/model"
    
train_model(model, datasets=tokenized_datasets, training_args=training_args, data_collator=data_collator, output_dir=model_output_dir)

tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
   
dataset = load_dataset("csv", data_files={"train": "../ch07/SST-2/train.tsv", "validation": "../ch07/SST-2/dev.tsv"}, delimiter='\t')
eval_model(model, dataset["validation"])
```

```bash
Acc: 0.8635321100917431
```

## 99. 選好チューニング


問題96のプロンプトに対して, 正解の感情ラベルを含むテキストを望ましい応答, 間違った感情ラベルを含むテキストを望ましくない応答として, 事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ.選好チューニングのアルゴリズムとしては, 近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる.

```python
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from trl import DPOConfig, DPOTrainer
from trl.trainer.utils import DPODataCollatorWithPadding
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm

set_seed(1234)

def tokenize_function(examples):
    tokenized_prompt = tokenizer(examples["prompt"], padding="max_length", truncation=True, max_length=128)
    tokenized_chosen = tokenizer(examples["chosen"], padding="max_length", truncation=True, max_length=128)
    tokenized_rejected = tokenizer(examples["rejected"], padding="max_length", truncation=True, max_length=128)
    return {
        "prompt_input_ids": tokenized_prompt["input_ids"],
        "chosen_input_ids": tokenized_chosen["input_ids"],
        "rejected_input_ids": tokenized_rejected["input_ids"],
    }

def make_pref_dataset(examples):
    records = {"prompt": [], "chosen": [], "rejected": []}
    for sent, lbl in zip(examples["sentence"], examples["label"]):
        prompt = f"Sentence: {sent}\nLabel:"
        chosen = f"{prompt} {lbl}"
        wrong_lbl = 1 - lbl
        rejected = f"{prompt} {wrong_lbl}"
        records["prompt"].append(prompt)
        records["chosen"].append(chosen)
        records["rejected"].append(rejected)
    return records

def train_model(model, train_dataset, eval_dataset, training_args, data_collator=None, compute_metrics=None, output_dir=None):
    trainer = DPOTrainer(
        model=model,
        args=training_args,
        processing_class=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )
    
    trainer.train()
    
    if output_dir is not None:
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

def eval_model(model, datasets, max_new_tokens=2, max_length=None, num_return_sequences=1, skip_special_tokens=True):
    input_texts = [f"Sentence: {ex['sentence']}\nLabel:" for ex in datasets]
    labels = [str(ex['label']) for ex in datasets]
    preds = []
    for input_text in tqdm(input_texts, desc="Evaluating"):
        inputs = tokenizer(input_text, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, max_length=max_length, num_return_sequences=num_return_sequences)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=skip_special_tokens, eos_token_id=tokenizer.eos_token_id)
        label_start = generated_text.find("Label:") + len("Label:")
        pred = generated_text[label_start:].strip().split()[0]
        preds.append(pred)
        
    acc = accuracy_score(labels, preds)
    print(f"Acc: {acc}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
        
tokenizer.pad_token = tokenizer.eos_token
       
dataset = load_dataset("csv", data_files={"train": "../ch07/SST-2/train.tsv", "validation": "../ch07/SST-2/dev.tsv"}, delimiter='\t')
train_pref_dataset = dataset["train"].map(make_pref_dataset, batched=True, remove_columns=dataset["train"].column_names).map(tokenize_function, batched=True)
eval_pref_dataset = dataset["validation"].map(make_pref_dataset, batched=True, remove_columns=dataset["validation"].column_names).map(tokenize_function, batched=True)

training_args = DPOConfig(
    output_dir="./dpo_results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    learning_rate=1e-5,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=500, 
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    beta=0.05,
    remove_unused_columns=True,
    disable_tqdm=False,
    eval_accumulation_steps=10
)

data_collator = DPODataCollatorWithPadding(
    pad_token_id=tokenizer.pad_token_id,
    label_pad_token_id=-100,
    is_encoder_decoder=False
)

model_output_dir = "./results/model_dop"
        
train_model(model, train_dataset=train_pref_dataset, eval_dataset=eval_pref_dataset, training_args=training_args, data_collator=data_collator, output_dir=model_output_dir)
 
tokenizer = AutoTokenizer.from_pretrained(model_output_dir)
model = AutoModelForCausalLM.from_pretrained(model_output_dir)
        
dataset = load_dataset("csv", data_files={"train": "../ch07/SST-2/train.tsv", "validation": "../ch07/SST-2/dev.tsv"}, delimiter='\t')
eval_model(model, dataset["validation"])
```

```bash
Acc: 0.9139908256880734
```